In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
# ID of the files on Google Drive
id = '1X0H850pVBBPctX4Z5otUn714_GLRWVtH'

# Download 'graph-full.txt'
graph_downloaded = drive.CreateFile({'id': id})
graph_downloaded.GetContentFile('graph-full.txt')

1a)

In [6]:
from pyspark import SparkConf, SparkContext
import re
from itertools import combinations
import numpy as np

# Set up Spark configuration
conf = SparkConf()
# Initialize Spark context
sc = SparkContext(conf=conf)


BETA = 0.8 # Damping factor for PageRank
MAX_ITER = 40 # Maximum number of iterations for PageRank
N = 1000 # Number of nodes in the graph

# Path to the input graph data file
path = "graph-full.txt"
# Read input data using Spark
data = sc.textFile(path)

In [7]:
# Load edges and combine the same edges
edges = (data.map(lambda line: tuple(map(int, re.split(r'\t+', line))))
         .groupByKey()
         .mapValues(lambda x: sorted(list(set(v for v in x))))
         .sortByKey())

# Obtain outgoing degrees
inv_d = edges.map(lambda x: 1 / len(x[1])).collect()

# Graph
graph = (edges.flatMapValues(lambda x: x)
         .map(lambda x: (x[1], x[0]))
         .groupByKey()
         .mapValues(lambda x: [(v, inv_d[v - 1]) for v in x])
         .sortByKey())

# Initialize r
N = edges.count()
r = [1 / N] * N

# PageRank
def pagerank(r, m):
    return m.mapValues(lambda x: sum([r[v[0] - 1] * v[1] * BETA for v in x]))\
            .mapValues(lambda x: x + (1 - BETA)/N)\
            .map(lambda lines: lines[1]).collect()

# Iterate
def iterate(r, m):
    for j in range(MAX_ITER):
        r = pagerank(r, m)
    return r

# Print Results
def find_and_print_top_bottom(r):
    r_sorted = sorted(r)
    r_array = np.array(r)

    top = [(np.where(r_array == r_sorted[-j - 1])[0][0] + 1, r_sorted[-j - 1]) for j in range(5)]
    bottom = [(np.where(r_array == r_sorted[j])[0][0] + 1, r_sorted[j]) for j in range(5)]

    print('Top:')
    for node_id, score in top:
        print(f'id: {node_id}, score: {score}')

    print('\nBottom:')
    for node_id, score in bottom:
        print(f'id: {node_id}, score: {score}')

r = iterate(r, graph)
find_and_print_top_bottom(r)

Top:
id: 263, score: 0.0020202911815182184
id: 537, score: 0.00194334157145315
id: 965, score: 0.001925447807166263
id: 243, score: 0.001852634016241731
id: 285, score: 0.0018273721700645144

Bottom:
id: 558, score: 0.0003286018525215297
id: 93, score: 0.0003513568937516577
id: 62, score: 0.00035314810510596274
id: 424, score: 0.00035481538649301454
id: 408, score: 0.00038779848719291705


1b)

In [8]:
# Load edges and combine the same edges
l = (data.map(lambda line: tuple(map(int, re.split(r'\t+', line))))
         .groupByKey()
         .mapValues(lambda x: sorted(list(set(v for v in x))))
         .sortByKey())

# Link matrix
def LT(rdd):
    return rdd.flatMapValues(lambda x: x)\
              .map(lambda x: (x[1], x[0]))\
              .groupByKey()\
              .mapValues(lambda x: [v for v in x])\
              .sortByKey()

# Compute a
def A(h, lt):
    return lt.mapValues(lambda x: sum([h[v-1] for v in x]))\
             .map(lambda lines: lines[1]).collect()

# Compute h
def H(a, l):
    return l.mapValues(lambda x: sum([a[v-1] for v in x]))\
            .map(lambda lines: lines[1]).collect()

# Iterate
def iterate_b(h, l, lt):
    for _ in range(MAX_ITER):
        a = A(h, lt)
        a_max = max(a)
        for i in range(len(a)):
            a[i] /= a_max
        h = H(a, l)
        h_max = max(h)
        for i in range(len(h)):
            h[i] /= h_max
    return a, h


# Find the high and low nodes
def high_and_low(r):
    r_sorted = sorted(r)
    r = np.array(r)
    low = []
    high = []
    for j in range(5):
        low.append((np.where(r == r_sorted[j])[0][0] + 1, r_sorted[j]))
        high.append((np.where(r == r_sorted[-j - 1])[0][0] + 1, r_sorted[-j - 1]))
    return high, low

# Print results
def print_results_b(high_a, low_a, high_h, low_h):
    print('High Hubbiness:')
    for i in range(5):
        print('id: ' + str(high_h[i][0]) + ', score: ' + str(high_h[i][1]))
    print('\nLow Hubbiness:')
    for i in range(5):
        print('id: ' + str(low_h[i][0]) + ', score: ' + str(low_h[i][1]))
    print('\nHigh Authority:')
    for i in range(5):
        print('id: ' + str(high_a[i][0]) + ', score: ' + str(high_a[i][1]))
    print('\nLow Authority:')
    for i in range(5):
        print('id: ' + str(low_a[i][0]) + ', score: ' + str(low_a[i][1]))

# Initialize h directly
h = [1] * N
# Compute Link Matrix (lt)
lt = LT(l)
# Iterate to compute 'a' and 'h'
a, h = iterate_b(h, l, lt)
# Find the high and low nodes for 'a' and 'h'
high_a, low_a = high_and_low(a)
high_h, low_h = high_and_low(h)
# Print the results
print_results_b(high_a, low_a, high_h, low_h)

High Hubbiness:
id: 840, score: 1.0
id: 155, score: 0.9499618624906543
id: 234, score: 0.8986645288972263
id: 389, score: 0.8634171101843789
id: 472, score: 0.8632841092495218

Low Hubbiness:
id: 23, score: 0.04206685489093652
id: 835, score: 0.057790593544330145
id: 141, score: 0.06453117646225177
id: 539, score: 0.0660265937341849
id: 889, score: 0.07678413939216452

High Authority:
id: 893, score: 1.0
id: 16, score: 0.9635572849634398
id: 799, score: 0.9510158161074015
id: 146, score: 0.9246703586198443
id: 473, score: 0.899866197360405

Low Authority:
id: 19, score: 0.05608316377607618
id: 135, score: 0.06653910487622794
id: 462, score: 0.07544228624641901
id: 24, score: 0.08171239406816942
id: 910, score: 0.08571673456144875
